# This is the data analysis test workbook

Goals include setting up data tables for each type of data

Plotting a line graph of the data

Allowing the person looking at the data to choose which county or country they wish to look at

In [ ]:
import numpy as np
import pandas as pd
import os
from pandas import DataFrame

In [ ]:
import webbrowser
website ='https://datahub.io/core/covid-19#data'
webbrowser.open(website)

In [ ]:
#finds the directory path for the file
dir_path = os.path.dirname(os.path.realpath('countries-aggregated.csv'))
dir_path

In [ ]:
#Finds all the files in the directory
f = []
for (dirpath, dirnames, filenames) in os.walk(dir_path):
    f.extend(filenames)
    break

In [ ]:
f

In [ ]:
files=[]
for Type in f:
    if Type.endswith('.csv'):
        files.append(Type)

In [ ]:
files

In [ ]:
dfs_names=[]
for x in files:
    n, q =x.split('.')
    dfs_names.append(n)

In [ ]:
dfs_names

In [ ]:
# Creats a dictionary with all my data frames within
dfs={}
for dfn,file in zip(dfs_names, files):
    dfs[dfn] = pd.read_csv(file)
    print(dfs[dfn].shape)
    print(dfs[dfn].dtypes)
dfs.keys()

In [ ]:
dfs["us_confirmed"]

In [ ]:
us_confirmed = dfs["us_confirmed"]

In [ ]:
Cal=us_confirmed[us_confirmed['Province/State']=='California']

In [ ]:
Cal_piv=Cal.pivot_table('Date','Admin2','Case')